In [30]:
import chess.pgn
import time
import sys
import math
import numpy as np

In [31]:
BOARD_SPACES = 64
ROWS = 8
COLS = 8

letter_lookup = {
    'a':0,
    'b':1,
    'c':2,
    'd':3,
    'e':4,
    'f':5,
    'g':6,
    'h':7
}
piece_lookup = {
    'r':0,
    'n':1,
    'b':2,
    'q':3,
    'k':4,
    'p':5,
    'R':6,
    'N':7,
    'B':8,
    'Q':9,
    'K':10,
    'P':11,
    '.':-1
}

In [32]:
# Format of Dataset
# One chanel per piece
# One additional channel showing whose turn it is
# Referenced from https://int8.io/chess-position-evaluation-with-convolutional-neural-networks-in-julia/

dataset_x = []
dataset_y = []
posns = []

pgn = open("datasets/AllGames.pgn")
game = chess.pgn.read_game(pgn)
black_or_white = True
count = 0
white_win = None

while game != None:
    # Get the game board
    board = game.board()

    # Truncate the number of games
    count += 1
    if count % 1000 == 0:
        print(count)
    if count >= 10000:
        break

    # Determine the winner of the game
    result = game.headers['Result']
    if result == '1-0':
        white_win = 1
    elif result == '0-1':
        white_win = 0
    else:
        white_win = 0.5
    num_moves = sum(1 for _ in game.mainline_moves())
            
    for i, move in enumerate(game.mainline_moves()):
        # Compute the board state
        state = 0.5*float(i+1)/num_moves
        if black_or_white == False:
            state *= -1
        state += 0.5        
        board.push(move)
                
        # Turn the board into a vector
        strboard = str(board)
        board_data = np.zeros((8, 8, 13))
        idx = 0
        for char in strboard:
            if char in piece_lookup:
                if char != '.':
                    piece = np.zeros(13)
                    piece[piece_lookup[char]] = 1
                    if black_or_white == True:
                        piece[-1] = 1
                    board_data[int(idx/COLS)][idx%ROWS] = piece
                idx += 1
                
        dataset_x.append(board_data)
        dataset_y.append(np.array(state))
        black_or_white = not black_or_white

    game = chess.pgn.read_game(pgn)

dataset_x = np.stack(dataset_x)
dataset_y = np.stack(dataset_y)

dataset_x[1], dataset_y[1]

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


(array([[[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
 
        [[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.

In [39]:
inshape = dataset_x[0].shape
outshape = dataset_y[0].size
print(dataset_x.shape, outshape)
print(dataset_y[0])

(856591, 8, 8, 13) 1
0.5142857142857142


In [40]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adadelta, Adam, RMSprop
from keras.callbacks import ModelCheckpoint, TensorBoard

In [41]:
model = Sequential()
model.add(Conv2D(128, kernel_size=(2, 2), padding='same',
                 input_shape=inshape))
model.add(Activation('relu'))

model.add(Conv2D(128, kernel_size=(2, 2), padding='same'))
model.add(Activation('relu'))

model.add(Flatten())

model.add(Dense(1024))
model.add(Activation('relu'))

model.add(Dense(1024))
model.add(Activation('relu'))

model.add(Dense(outshape, activation='sigmoid'))

model.compile(loss=keras.losses.mean_squared_error,
              optimizer=Adam(lr=0.001),
              metrics=['mse'])

tensorboard = TensorBoard(log_dir='logs')
filepath = '/home/mitchellcatoen/weights.{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, verbose=1)
callbacks = [tensorboard, checkpoint]

# model.load_weights(filepath)

In [42]:
# import sys
# sys.stdout = open("output.txt", "w")
# print("Starting")

In [ ]:
model.fit(dataset_x, dataset_y, epochs=200, batch_size=64, callbacks=callbacks, validation_split=0.1)

Train on 770931 samples, validate on 85660 samples
Epoch 1/200
770931/770931 [==============================] - 249s 323us/step - loss: 0.0036 - mean_squared_error: 0.0036 - val_loss: 0.0019 - val_mean_squared_error: 0.0019

Epoch 00001: saving model to /home/mitchellcatoen/weights.01-0.00.hdf5
Epoch 2/200
770931/770931 [==============================] - 249s 323us/step - loss: 0.0011 - mean_squared_error: 0.0011 - val_loss: 0.0010 - val_mean_squared_error: 0.0010

Epoch 00002: saving model to /home/mitchellcatoen/weights.02-0.00.hdf5
Epoch 3/200
770931/770931 [==============================] - 249s 323us/step - loss: 7.3461e-04 - mean_squared_error: 7.3461e-04 - val_loss: 7.5690e-04 - val_mean_squared_error: 7.5690e-04

Epoch 00003: saving model to /home/mitchellcatoen/weights.03-0.00.hdf5
Epoch 4/200
770931/770931 [==============================] - 249s 323us/step - loss: 5.8658e-04 - mean_squared_error: 5.8658e-04 - val_loss: 6.5641e-04 - val_mean_squared_error: 6.5641e-04

Epoch 00

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



770931/770931 [==============================] - 249s 323us/step - loss: 4.5127e-04 - mean_squared_error: 4.5127e-04 - val_loss: 5.5476e-04 - val_mean_squared_error: 5.5476e-04

Epoch 00006: saving model to /home/mitchellcatoen/weights.06-0.00.hdf5
Epoch 7/200
  8704/770931 [..............................] - ETA: 4:04 - loss: 4.0718e-04 - mean_squared_error: 4.0718e-04

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



770931/770931 [==============================] - 250s 324us/step - loss: 4.1699e-04 - mean_squared_error: 4.1699e-04 - val_loss: 4.9544e-04 - val_mean_squared_error: 4.9544e-04

Epoch 00007: saving model to /home/mitchellcatoen/weights.07-0.00.hdf5
Epoch 8/200
157888/770931 [=====>........................] - ETA: 3:14 - loss: 3.8137e-04 - mean_squared_error: 3.8137e-04

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



770931/770931 [==============================] - 249s 323us/step - loss: 2.4492e-04 - mean_squared_error: 2.4492e-04 - val_loss: 3.3310e-04 - val_mean_squared_error: 3.3310e-04

Epoch 00048: saving model to /home/mitchellcatoen/weights.48-0.00.hdf5
Epoch 49/200
 85504/770931 [==>...........................] - ETA: 3:37 - loss: 2.4632e-04 - mean_squared_error: 2.4632e-04

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



770931/770931 [==============================] - 250s 324us/step - loss: 2.4469e-04 - mean_squared_error: 2.4469e-04 - val_loss: 3.3008e-04 - val_mean_squared_error: 3.3008e-04

Epoch 00049: saving model to /home/mitchellcatoen/weights.49-0.00.hdf5
Epoch 50/200
770931/770931 [==============================] - 249s 324us/step - loss: 2.4348e-04 - mean_squared_error: 2.4348e-04 - val_loss: 3.3457e-04 - val_mean_squared_error: 3.3457e-04

Epoch 00050: saving model to /home/mitchellcatoen/weights.50-0.00.hdf5
Epoch 51/200
770931/770931 [==============================] - 250s 324us/step - loss: 2.4325e-04 - mean_squared_error: 2.4325e-04 - val_loss: 3.4136e-04 - val_mean_squared_error: 3.4136e-04

Epoch 00051: saving model to /home/mitchellcatoen/weights.51-0.00.hdf5
Epoch 52/200
770931/770931 [==============================] - 248s 322us/step - loss: 2.4131e-04 - mean_squared_error: 2.4131e-04 - val_loss: 3.3798e-04 - val_mean_squared_error: 3.3798e-04

Epoch 00052: saving model to /home/m

770931/770931 [==============================] - 237s 307us/step - loss: 2.2716e-04 - mean_squared_error: 2.2716e-04 - val_loss: 3.1239e-04 - val_mean_squared_error: 3.1239e-04

Epoch 00080: saving model to /home/mitchellcatoen/weights.80-0.00.hdf5
Epoch 81/200
770931/770931 [==============================] - 237s 307us/step - loss: 2.2819e-04 - mean_squared_error: 2.2819e-04 - val_loss: 3.1202e-04 - val_mean_squared_error: 3.1202e-04

Epoch 00081: saving model to /home/mitchellcatoen/weights.81-0.00.hdf5
Epoch 82/200
770931/770931 [==============================] - 236s 306us/step - loss: 2.2882e-04 - mean_squared_error: 2.2882e-04 - val_loss: 3.6600e-04 - val_mean_squared_error: 3.6600e-04

Epoch 00082: saving model to /home/mitchellcatoen/weights.82-0.00.hdf5
Epoch 83/200
770931/770931 [==============================] - 237s 307us/step - loss: 2.2625e-04 - mean_squared_error: 2.2625e-04 - val_loss: 3.1035e-04 - val_mean_squared_error: 3.1035e-04

Epoch 00083: saving model to /home/m

In [28]:
score = model.evaluate(dataset_x, dataset_y, batch_size=64)
print(score)

7110/7110 [==============================] - 0s 61us/step
[0.0015044477707988016, 0.0015044477707988016]


In [29]:
for i in range(10):
    output = model.predict(np.reshape(dataset_x[i], (1, 8, 8, 13)))
    print(output, dataset_y[i])

[[0.50590014]] 0.5142857142857142
[[0.48962843]] 0.4714285714285714
[[0.5254903]] 0.5428571428571428
[[0.47565675]] 0.44285714285714284
[[0.540834]] 0.5714285714285714
[[0.45788556]] 0.41428571428571426
[[0.56273574]] 0.6
[[0.4388967]] 0.38571428571428573
[[0.6112258]] 0.6285714285714286
[[0.4060839]] 0.35714285714285715


In [ ]:
'''Trains a simple convnet on the MNIST dataset.
Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print(input_shape)

# # convert class vectors to binary class matrices
# y_train = keras.utils.to_categorical(y_train, num_classes)
# y_test = keras.utils.to_categorical(y_test, num_classes)

# model = Sequential()
# model.add(Conv2D(32, kernel_size=(3, 3),
#                  activation='relu',
#                  input_shape=input_shape))
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(num_classes, activation='softmax'))

# model.compile(loss=keras.losses.categorical_crossentropy,
#               optimizer=keras.optimizers.Adadelta(),
#               metrics=['accuracy'])

# model.fit(x_train, y_train,
#           batch_size=batch_size,
#           epochs=epochs,
#           verbose=1,
#           validation_data=(x_test, y_test))
# score = model.evaluate(x_test, y_test, verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

# # print(x_train[0])
# # print(y_train.shape)

In [ ]:
import os
cwd = os.getcwd()
cwd

In [ ]:
dataset_y[0]